In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# Fotrain = pd.read_csv("train.csv")r example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output

# Any results you write to the current directory are saved as output.
import tensorflow as tf
from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [13]:
#train = pd.read_csv("train.csv")

In [14]:
#train

In [15]:
#test = pd.read_csv("test.csv")

In [16]:
#test

In [2]:
max_features = 20000
maxlen = 100

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.sample(frac=1)  #这一步是把顺序打乱，如果想恢复成正常顺序：df = df.sample(frac=1).reset_indedx(drop=True)

list_sentences_train = train["comment_text"].fillna("CVxTz").values 
#fillna是吧所有的NA用“CVxTz”进行填充， 之后的.value是用来提取train["comment_text"]填充之后的所有的语句
#提取之后的输出是这样的：每一行是一个句子

'''
[ 'Silly games? \n\nIm sorry but what did I DO???????????????????????????????????????????????????????'
 '"\n\n Surname \n\nHow do you pronounce it?  talk "'
 "I also just noticed: he simultaneously went after these same articles on the french Wikipedia, where he has ALSO been banned. See , for example. As I've said before, this editor simply refuses to give up, no matter how many warnings he is given."
 ...,
 "Thank you for correcting my typo in Thomas Pooley. I am the world's worst proofreader!"
 '(most of which link to a clip of goths beating up emo kids)'
 '":You seem to know a lot on this subject!! Have you checked out any of those links?  Smith \n\n"']

'''

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values   #这部分就像是training set的label
#同样是提取value：提取之后是这样
'''
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 ..., 
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
'''

list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [19]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.sample(frac=1)  #这一步是把顺序打乱，如果想恢复成正常顺序：df = df.sample(frac=1).reset_indedx(drop=True)
list_sentences_train = train["comment_text"].fillna("CVxTz").values 

In [20]:
list_sentences_train.shape

(159571,)

In [21]:
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values   #这部分就像是training set的label
#同样是提取value：提取之后是这样

In [22]:
y.shape

(159571, 6)

In [23]:
list_sentences_test = test["comment_text"].fillna("CVxTz").values

In [24]:
list_sentences_test.shape

(153164,)

In [25]:
max_features = 20000
maxlen = 100
tokenizer = text.Tokenizer(num_words=max_features)  #将文本转化成向量
tokenizer.fit_on_texts(list(list_sentences_train))  #先将array转换成list，fit_on_texts里面是要用以训练的文本列表
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)  #之后转换成sequence
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

#将所有的长序列转化成等长序列 
#如果提供了参数maxlen，nb_timesteps=maxlen，否则其值为最长序列的长度。其他短于该长度的序列都会在后部填充0以达到该长度
X_t = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)   #一开始定义了maxlen=100
X_te = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)
#可以print一下所有的句子都被向量化并且等长

In [1]:
X_t[0]

NameError: name 'X_t' is not defined

In [41]:
X_t[0].shape

(100,)

In [ ]:
from keras.models import Sequential


model = Sequential()
model.add(Embedding(1000, 64, input_length=10))



In [8]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)    #(20000,128)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)   #全连接
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)   #全连接
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [9]:
model = get_model()
batch_size = 32
epochs = 2


file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

early = EarlyStopping(monitor="val_loss", mode="min", patience=20)


callbacks_list = [checkpoint, early] #early
model.fit(X_t, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

model.load_weights(file_path)

y_test = model.predict(X_te)



Train on 143613 samples, validate on 15958 samples
Epoch 1/2
143613/143613 [==============================] - 986s 7ms/step - loss: 0.0664 - acc: 0.9783 - val_loss: 0.0494 - val_acc: 0.9827
Epoch 2/2
143613/143613 [==============================] - 1000s 7ms/step - loss: 0.0464 - acc: 0.9828 - val_loss: 0.0473 - val_acc: 0.9833


In [17]:
np.save('y_test.npy',y_test)


In [20]:
aaa = np.load('y_test.npy')
print(np.array(aaa))

[[  9.98250306e-01   4.15136218e-01   9.89175141e-01   5.72846793e-02
    8.76540124e-01   1.27772570e-01]
 [  2.48284778e-04   6.22008329e-08   2.67030155e-05   3.25830342e-07
    2.69377360e-05   3.08155245e-06]
 [  1.79661333e-03   1.92765651e-06   2.85605318e-04   1.07817805e-05
    2.37976055e-04   5.32493214e-05]
 ..., 
 [  3.31642048e-04   4.97843544e-08   2.67797750e-05   1.37724044e-07
    3.05294961e-05   1.94226232e-06]
 [  7.46669422e-04   1.51656039e-07   5.87371032e-05   6.22478296e-07
    6.65541666e-05   6.78119113e-06]
 [  9.87695098e-01   1.01110503e-01   9.23692524e-01   2.58676950e-02
    7.22269475e-01   7.98792988e-02]]


In [19]:
y_test

array([[  9.98250306e-01,   4.15136218e-01,   9.89175141e-01,
          5.72846793e-02,   8.76540124e-01,   1.27772570e-01],
       [  2.48284778e-04,   6.22008329e-08,   2.67030155e-05,
          3.25830342e-07,   2.69377360e-05,   3.08155245e-06],
       [  1.79661333e-03,   1.92765651e-06,   2.85605318e-04,
          1.07817805e-05,   2.37976055e-04,   5.32493214e-05],
       ..., 
       [  3.31642048e-04,   4.97843544e-08,   2.67797750e-05,
          1.37724044e-07,   3.05294961e-05,   1.94226232e-06],
       [  7.46669422e-04,   1.51656039e-07,   5.87371032e-05,
          6.22478296e-07,   6.65541666e-05,   6.78119113e-06],
       [  9.87695098e-01,   1.01110503e-01,   9.23692524e-01,
          2.58676950e-02,   7.22269475e-01,   7.98792988e-02]], dtype=float32)

In [10]:
sample_submission = pd.read_csv("sample_submission.csv")

sample_submission[list_classes] = y_test

sample_submission.to_csv("results2.csv", index=False)

In [13]:
results2 = pd.read_csv("results2.csv")

print(type(results2['id'][0]))

<class 'str'>
